# Regressor for Market Sales

### Authors: Giacomo Bossi,  Emanuele Chioso


## Abstract

The goal of the project is to provide a working
forecasting model to optimize promotions and
warehouse stocks of one of the most important
European retailers

## Approach

We started analysing the dataset we were given,
trying to identify correlations or patterns
between features. Once the data analysis was
complete we cleaned it (as explained in the next
section).
We then proceeded to implement some basic
regressor algorithms in order to have a first
glance of what the general performance on the
dataset was using R2 score and MAE as the evaluation
metric.
In the end we selected a few of them and
ensembled their predictions to obtain the final
prediction for the test set.
All testing was performed via holdout testing to
get a quick result for completely new classifiers,
and later with cross validation to get a less
randomized evaluation.

#### Importation of all useful packets

In [ ]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold='nan')
from sklearn import linear_model
from sklearn import model_selection
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import f_classif
from datetime import datetime
from scipy.special import boxcox1p
from scipy import stats
from scipy.stats import norm,skew
import warnings


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
%matplotlib inline
pd.set_option('display.max_rows', 500)

#### Functions defined in Preprocessing

In [ ]:
def search_log_to_skewed_features(df):
    numeric_feats = []
    for col in df.columns:
        if(len(df[col].unique())>2 and df[col].dtype != "object"):
            numeric_feats.append(col)
    
    # Check the skew of all numerical features
    skewed_feats = df[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
    skewness = pd.DataFrame({'Skew': skewed_feats})
    skewness = skewness[abs(skewness.Skew) > 0.75]
    skewed_features = skewness.index    
    return skewed_features

def apply_log_to_skewed_features(df,skewed_features,lambda_log = 0.15):
    for feat in skewed_features:
        df[feat] = boxcox1p(df[feat], lambda_log)
    print("logged features:",skewed_features)
    return df

def apply_exp_to_result(df,lambda_log = 0.15):
    print(df[target].mean())
    df[feat] = np.inv_boxcox1p(df[target], lambda_log)
    print(df[target].mean())
    return df

def add_date(df):
    date = np.array(  [ x.split('/') for x in df['Date'].values])
    date = date.astype(np.int32)
    df['Date'] = [ datetime(x[2],x[1],x[0])  for x in date ]
    
def apply_exp_to_result(df,lambda_log = 0.15):
    return inv_boxcox1p(df, lambda_log)

# Data Acquisition

In [ ]:
dataset = pd.read_csv('../data/original_train.csv') 
testset = pd.read_csv('../data/original_test.csv')

dataset=dataset[dataset.IsOpen != 0]
testset=testset[testset.IsOpen != 0]


components = dataset.columns[dataset.columns!='Unnamed: 0']
tcomponents = testset.columns[testset.columns!='Unnamed: 0']

features=set(components).intersection(tcomponents)
wtarget=list(set(components)-set(tcomponents))
target = 'NumberOfSales'

# Dealing with NAN
We have substituted the missing values in
Max_Gust_Speed with the values of Max_Wind.
Then, in order to fill all the missing values, we
have grouped the dataset by the StoreID and
after that, we have used a linear interpolation
taking as index the time feature.Since the
missing values of ‘Events’ are NMAR we haven’t
handle it.
## Dataset

In [ ]:
add_date(dataset)
for val in dataset['StoreID'].unique():
    df = pd.DataFrame(dataset.loc[dataset['StoreID'] == val])
    df.index = df['Date']
    
    df['tOpen']=df['IsOpen'].shift(-1).fillna(method='ffill')
    df['yOpen']=df['IsOpen'].shift(+1).fillna(method='bfill')
    df['tPromotions']=df['HasPromotions'].shift(-1).fillna(method='ffill')
    df['yPromotions']=df['HasPromotions'].shift(+1).fillna(method='bfill')
    df = df.interpolate(method='time',downcast='infer',limit=10)
    
    
    
    dataset.drop(dataset.loc[dataset['StoreID'] == val].index, inplace=True)
    df.index = df['StoreID']
    dataset = pd.concat([dataset, df],ignore_index=True)

    
dataset['Precipitationmm'] = (np.ceil(dataset.Precipitationmm / 10) * 1).astype(int)
dataset['CloudCover'] = dataset['CloudCover'].fillna(dataset['Precipitationmm'])
dataset['Max_Gust_SpeedKm_h'] = dataset['Max_Gust_SpeedKm_h'].fillna(dataset['Max_Wind_SpeedKm_h'])

#Convert some data to integer
col_to_int = ['Min_VisibilitykM','Max_VisibilityKm','Max_Gust_SpeedKm_h',
              'CloudCover','Mean_VisibilityKm','HasPromotions','IsHoliday','HasPromotions']
for col in col_to_int:
    dataset[col] = dataset[col].astype(int)

#Convert some data to int since they are One Hot Encoded
    
#Add some datas about time
dataset['Month'] = pd.DatetimeIndex(dataset['Date']).month
dataset['Daysmonth']= pd.DatetimeIndex(dataset['Date']).day
dataset['Daysweek']= pd.DatetimeIndex(dataset['Date']).dayofweek
dataset['Quarter']= pd.DatetimeIndex(dataset['Date']).quarter
dataset['Year']= pd.DatetimeIndex(dataset['Date']).year


dataset.drop(columns='Date', inplace=True)
dataset.drop(columns='IsOpen', inplace=True)

## Testset

In [ ]:
add_date(testset)


for val in testset['StoreID'].unique():
    print(val,testset.shape)
    df = pd.DataFrame(testset.loc[testset['StoreID'] == val])
    df.index = df['Date']
    
    df['tOpen']=df['IsOpen'].shift(-1).fillna(method='ffill')
    df['yOpen']=df['IsOpen'].shift(+1).fillna(method='bfill')
    df['tPromotions']=df['HasPromotions'].shift(-1).fillna(method='ffill')
    df['yPromotions']=df['HasPromotions'].shift(+1).fillna(method='bfill')
    df = df.interpolate(method='time',downcast='infer', limit=100)
    
    
    testset.drop(testset.loc[testset['StoreID'] == val].index, inplace=True)
    df.index = df['StoreID']
    print(val,df.shape)

    testset = pd.concat([testset, df],ignore_index=True)
    print(val,testset.shape)
print(testset.shape)
testset['Precipitationmm'] = (np.ceil(testset.Precipitationmm / 10) * 1).astype(int)
testset['CloudCover'] = testset['CloudCover'].fillna(testset['Precipitationmm'])
testset['Max_Gust_SpeedKm_h'] = testset['Max_Gust_SpeedKm_h'].fillna(testset['Max_Wind_SpeedKm_h'])


In [ ]:
testset['Min_VisibilitykM']=testset['Min_VisibilitykM'].fillna(testset['Min_VisibilitykM'].mean())
testset['Max_VisibilityKm']=testset['Max_VisibilityKm'].fillna(testset['Max_VisibilityKm'].mean())
testset['Mean_VisibilityKm']=testset['Mean_VisibilityKm'].fillna(testset['Mean_VisibilityKm'].mean())

#Convert some data to integer
col_to_int = ['Min_VisibilitykM','Max_VisibilityKm','Max_Gust_SpeedKm_h',
              'CloudCover','Mean_VisibilityKm','HasPromotions','IsHoliday',
              'Region','Region_AreaKM2','Region_GDP','Region_PopulationK']
for col in col_to_int:
    testset[col] = testset[col].astype(int)

#Add some datas about time
testset['Month'] = pd.DatetimeIndex(testset['Date']).month
testset['Daysmonth']= pd.DatetimeIndex(testset['Date']).day
testset['Daysweek']= pd.DatetimeIndex(testset['Date']).dayofweek
testset['Quarter']= pd.DatetimeIndex(testset['Date']).quarter
testset['Year']= pd.DatetimeIndex(testset['Date']).year

testset.drop(columns='Date', inplace=True)
testset.drop(columns='IsOpen', inplace=True)

### Check the remained missing data

In [ ]:
train_tmp = (testset.isnull().sum() / len(testset)) * 100
train_tmp = train_tmp.drop(train_tmp[train_tmp == 0].index).sort_values(ascending=False)[:100]
missing_data = pd.DataFrame({'Missing Ratio' :train_tmp})

# PCA Analysis and Reduction
## Weather Features
In order to reduce the number of parameters
bound to the weather features and augment the
information associated with a single feature we
have performed a Principal Component
Analysis.
We can see in this Heatmap the strong
correlations between the weather features
Considering only the first 4 components we
have reached a cumulative variance of ~98%.
So, we have reduced 20 different features into 4,
loosing only a 2% of information. Before and
after the PCA we have also performed a
normalization of the parameters to attenuate
the sensibility of this analysis to scale.

In [ ]:
wheather_features = ['Max_Humidity', 'Max_Sea_Level_PressurehPa', 'Max_TemperatureC',
       'Max_VisibilityKm', 'Max_Wind_SpeedKm_h', 'Mean_Dew_PointC',
       'Mean_Humidity', 'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC','CloudCover',
       'Mean_VisibilityKm', 'Mean_Wind_SpeedKm_h', 'Min_Dew_PointC', 'Max_Dew_PointC', 
       'Min_Humidity', 'Min_Sea_Level_PressurehPa', 'Min_TemperatureC',
       'Min_VisibilitykM', 'Precipitationmm', 'WindDirDegrees','Max_Gust_SpeedKm_h']
full_pca_model = PCA()

n_dataset = dataset.shape[0]
n_testset = testset.shape[0]

superset = pd.concat([dataset,testset]).reset_index(drop=True)
superset[wheather_features] = preprocessing.normalize(superset[wheather_features])

full_fitted_model = full_pca_model.fit(superset[wheather_features])


corr = superset[weather_features].corr()
plt.subplots(figsize=(12,9))

plt.figure(figsize=(8, 8))
plt.semilogy(full_fitted_model.explained_variance_ratio_, '--o')
plt.xticks(np.arange(0,len(wheather_features),1))
plt.xlabel("Features")
plt.ylabel("Explained Variance Ratio")

plt.figure(figsize=(12, 12))
plt.semilogy(full_fitted_model.explained_variance_ratio_.cumsum(), '--o')
plt.xticks(np.arange(0,len(wheather_features),1))
plt.xlabel("Features")
plt.ylabel("Cumulative Explained Variance Ratio")


PCA_components=4
feature_selection_pca_model = PCA(n_components=PCA_components, svd_solver='full')

fitted_model = feature_selection_pca_model.fit(superset[wheather_features])
X_selected_features_pca = fitted_model.transform(superset[wheather_features])

toAdd = pd.DataFrame(X_selected_features_pca) 
preprocessing.normalize(toAdd,axis=0)
for i in range(0,PCA_components):
    superset['wheather_PCA_'+str(i)]= toAdd[i]
superset.drop(columns=wheather_features, inplace=True)

## Region
We have performed the same transformation
even to the features of the region. We have
reduced the four features of a region into 2
features, loosing less than 4% of variance.

In [ ]:
#reduce the number of region features
region_features = ['Region_AreaKM2','Region_GDP','Region_PopulationK']

superset[region_features] = preprocessing.normalize(superset[region_features])

full_fitted_model = full_pca_model.fit(superset[region_features])

corr = superset[region_features].corr()
plt.subplots(figsize=(12,9))

plt.figure(figsize=(8, 8))
plt.semilogy(full_fitted_model.explained_variance_ratio_, '--o')
plt.xticks(np.arange(0,len(region_features),1))
plt.xlabel("Features")
plt.ylabel("Explained Variance Ratio")

plt.figure(figsize=(12, 12))
plt.semilogy(full_fitted_model.explained_variance_ratio_.cumsum(), '--o')
plt.xticks(np.arange(0,len(region_features),1))
plt.xlabel("Features")
plt.ylabel("Cumulative Explained Variance Ratio")

PCA_components=2
feature_selection_pca_model = PCA(n_components=PCA_components, svd_solver='full')

fitted_model = feature_selection_pca_model.fit(superset[region_features])
X_selected_features_pca = fitted_model.transform(superset[region_features])


toAdd = pd.DataFrame(X_selected_features_pca) 
preprocessing.normalize(toAdd,axis=0)

for i in range(0,PCA_components):
    superset['region_PCA_'+str(i)]= toAdd[i]
superset.drop(columns=region_features, inplace=True)

# OHE One Hot Encoding

In [ ]:
##EXCEPTION FOR DAYS AND MONTHS
for col in superset.columns:
    if (superset[col].dtypes == 'object'):
        for elem in superset[col].unique():
            elem = str(elem)
            superset[col+'_'+elem] = superset[col].apply(lambda x: 1 if str(x)==elem else 0).values.astype(float)
        superset.drop(columns=col,inplace=True)
        
dataset = superset[:n_dataset]
testset = superset[n_dataset:]

# Distibution of the Target
## Skewness removing
After some analysis, we have noticed that some
variables and also the target were skewed. So,
trying to fit a gaussian distribution we have
noticed some differences. As we notice below
for the target variable, the distribution of the
target was right-skewed.

In [ ]:

plt.figure(figsize=(8, 8))

sns.distplot(dataset['NumberOfSales'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(dataset['NumberOfSales'])

print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution

plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('NumberOfSales distribution')


#Get also the QQ-plot
fig = plt.figure()
plt.figure(figsize=(8, 8))

res = stats.probplot(dataset['NumberOfSales'], plot=plt)
plt.show()


In [ ]:
sk_feat = search_log_to_skewed_features(dataset)
dataset = apply_log_to_skewed_features(dataset,sk_feat)
sk_feat = set(sk_feat)-set(['NumberOfSales', 'NumberOfCustomers'])
testset = apply_log_to_skewed_features(testset,sk_feat)

So, we have decided to apply the log
transformation to all the variables that had a
skewness greater than 0,75. The result obtained
for the target are the following:

In [ ]:
plt.figure(figsize=(8, 8))

sns.distplot(dataset['NumberOfSales'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(dataset['NumberOfSales'])

print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution

plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('NumberOfSales distribution')


#Get also the QQ-plot
fig = plt.figure()
plt.figure(figsize=(8, 8))

res = stats.probplot(dataset['NumberOfSales'], plot=plt)
plt.show()


# Correlation Analysis

In [ ]:
corr = dataset.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corr, vmax=0.9, square=True)

# Feature Selection
## Random Forest Selection
To select the best features found during the preprocessing we have done several features
selection, as PCA feature selection, Correlation
based features selection and Random Forest
features selection. Since the best model found
was a XGBoost we have used a Random Forest
features selection. The threshold was set at 2 ∙ Median,
in order to
take all the features before the step in the
middle (~0,02). So, we have selected the first 21
features.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split

components = dataset.columns#[dataset.dtypes != 'object']
features=list(set(components) - set(wtarget))
#dataset[features] = dataset[features].values.astype(float)

cv = KFold(n_splits=2, random_state=21)

X = np.array(dataset[features])
y = np.array(dataset[target])
selected_feat = dataset[features].columns


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
forest = ExtraTreesRegressor(n_estimators=250, random_state=0, n_jobs=-1)
forest.fit(X, y)

importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], selected_feat[indices[f]], importances[indices[f]]))
    
# Plot the feature importances of the forest
plt.figure()
plt.figure(figsize=(12, 12))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), selected_feat[indices],rotation=90)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
from sklearn.feature_selection import SelectFromModel
feature_selection_model = SelectFromModel(forest, prefit=True,threshold='1.5*median')
X_selected_features_forest = feature_selection_model.transform(X)

In [ ]:
X_selected_features_forest.shape

In [ ]:
X_test = np.array(testset[features])
X_test_selected_features_forest = feature_selection_model.transform(X_test)

In [ ]:
np.save('X.npy',X)
np.save('y.npy',y)
np.save('X_selected.npy',X_selected_features_forest)
np.save('X_test.npy',X_test)
np.save('X_test_selected.npy',X_test_selected_features_forest)

# Model Selection and Evaluation
We have trained several different models, in
order to have a more reliable valuation of the
best model to use. First of all, we have trained a
simple model, KNN regressor

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.special import boxcox1p, inv_boxcox1p
from tqdm import tqdm
import xgboost as xgb
import lightgbm as lgb
import pickle
import numpy as np 

### Lasso

In [ ]:
lasso_params = { 'alpha':5e-02 }
lasso =  Lasso(max_iter=10000, **lasso_params)

### Light Boost Parameters

In [ ]:
lgb_params = {'n_jobs': -1, 
              'min_child_w': 1, 
              'colsample': 0.5, 
              'bagging_seed': 10, 
              'learning_rate': 0.7, 
              'bagging_fraction': 1, 
              'min_data_in_leaf': 8,
              'objective': 'regression', 
              'num_leaves': 400, 
              'estimators': 100, 
              'bagging_freq': 1, 
              'reg_lambda': 0.9, 
              'reg_alpha': 0.9,
              'max_bin': 300, 
              'min_sum_hessian_in_leaf': 11}

model_lgb = lgb.LGBMRegressor(**lgb_params)

### XGBoost Parameters

In [ ]:
xgb_params ={
            "n_estimators":100,
             "colsample":0.5,
             "gamma":0.05,
             "learning":0.1,
             "max_dep":30,
             "min_child_w":1,
             "reg_alpha":0.9,
             "reg_lambda":0.8,
             "n_jobs":-1 }

xgb_params2 ={
            "n_estimators":50,
             "colsample":0.5,
             "gamma":0.05,
             "learning":0.1,
             "max_dep":30,
             "min_child_w":1,
             "reg_alpha":0.9,
             "reg_lambda":0.8,
             "n_jobs":-1 }

model_xgb = xgb.XGBRegressor(**xgb_params)
model_xgb2 = xgb.XGBRegressor(**xgb_params2)

### Random Forest Parameters

In [ ]:
forest_params = {'min_impurity_decrease': False, 'max_features': 'auto', 'oob_score': False, 'bootstrap': True, 
                     'warm_start': False, 'n_jobs': -1, 'criterion': 'mse', 'min_weight_fraction_leaf': 1e-07, 
                     'min_samples_split': 5, 'min_samples_leaf': 1, 'max_leaf_nodes': None, 'n_estimators': 50, 
                     'max_depth': 50}

model_forest = RandomForestRegressor(**forest_params)

### Lasso Score

In [ ]:
lasso_preds = lasso_model.predict(X_test) 
print("SCORE:", r2_score(y_test, apply_exp_to_result(lasso_preds)))

### KNN Score
The first model trained, in order to have a
baseline to overreach was the KNN. We have
trained this model with a different number of
neighbours and the best result we have
obtained was: R2 Score ≅ 0.68, using a 10 folds
cross validation.

In [ ]:
result=[]
kfolds = KFold(10,shuffle=True,random_state=1234)

for i in range(2,30,1):
    neigh = KNeighborsRegressor(n_neighbors=i)
    scores = cross_val_score(neigh, X_selected_features_forest, y, cv=kfolds)
    print('KNN has obtained',scores.mean(),'with number of Neighboors=',i)
    result.append((i,scores.mean()))
plt.figure(figsize=(12,12))
results = pd.DataFrame(result)
plt.plot(results[0], results[1] ,linestyle='-', marker=".", color='green', markersize=3, label="R2")

### LightBoost Score

In [ ]:
model_lgb.fit(X,y)
lgb_preds = model_lgb2.predict(X_test) 

print("SCORE:", r2_score(y_test, apply_exp_to_result(lgb_preds)))

### Random Forest Score

In [ ]:
model_forest.fit(X,y)
forest_preds = model_forest.predict(X_test) 

print("SCORE:", r2_score(y_test, apply_exp_to_result(forest_preds)))

### XGB Sore

In [ ]:
model_xgb.fit(X,y)
xgb_preds = model_xgb.predict(X_test) 

print("SCORE:", r2_score(y_test, apply_exp_to_result(xgb_preds)))

### Model Averaging

In [ ]:
mean_results = (lgb_preds+forest_preds+xgb_preds)/3
print("SCORE:", r2_score(y_test, apply_exp_to_result(mean_results))
print("SCORE:", mean_absolute_error(y_test, apply_exp_to_result(mean_results))

# Model Ensembling
Finally, we have tried to use metamodeling
since the averaging of base model improves the
results. In this approach, we have created a
meta model based on average base models and
used an out-of-folds prediction of these models
to train out meta model. Since the best base
model were: Random Forest, LightBoost, XGBoost.
The final model is the result of an ensemble of the single models.
The models performed very well with trainset created with a Random Sampling but in a more realistic approach, where we predicted two entire months, they have been outperformed by the ensembles.

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=10):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (model_xgb, model_lgb, model_forest),
                                                 meta_model = model_xgb2)
stacked_averaged_models.fit(X,y)
averaged_models_preds = stacked_averaged_models.predict(X_test)
averaged_models_preds = apply_exp_to_result(averaged_models_preds)
print("R2 Score:", r2_score(y_train, averaged_models_preds))
print("MAE Score:", mean_absolute_error(y_train, averaged_models_preds))